In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("CarPrice.csv")
df

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
CompanyName=df['CarName'].apply(lambda x: x.split(' ')[0])
df.insert(3,'CompanyName',CompanyName)
df.drop(['CarName'],axis=1,inplace=True)
df.head()

In [ ]:
sns.distplot(df['price'])
plt.show()

In [ ]:
plt.figure(figsize=(16,8))

df1 = df.select_dtypes(include=['float64', 'int'])
df1 = df1.drop(['symboling', 'car_ID'], axis=1)
df1 = df1.corr()

sns.heatmap(df1, cmap="RdYlBu", annot=True)
plt.show()

In [ ]:
df['fueleconomy'] = (0.55 * df['citympg']) + (0.45 * df['highwaympg'])
df['price'] = df['price'].astype('int')
temp = df.copy()
table = temp.groupby(['CompanyName'])['price'].mean()
temp = temp.merge(table.reset_index(), how='left',on='CompanyName')
bins = [0,10000,20000,40000]
cars_bin=['Budget','Medium','Highend']
df['carsrange'] = pd.cut(temp['price_y'],bins,right=False,labels=cars_bin)
df.head()

In [ ]:
cars_lr = df[['price', 'fueltype', 'aspiration','carbody', 'drivewheel','wheelbase',
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 
                    'fueleconomy', 'carlength','carwidth', 'carsrange']]
cars_lr.head()

In [ ]:
def preds(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df

cars_lr = preds('fueltype',cars_lr)
cars_lr = preds('aspiration',cars_lr)
cars_lr = preds('carbody',cars_lr)
cars_lr = preds('drivewheel',cars_lr)
cars_lr = preds('enginetype',cars_lr)
cars_lr = preds('cylindernumber',cars_lr)
cars_lr = preds('carsrange',cars_lr)

In [ ]:
y=cars_lr.pop('price')
x=cars_lr
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)
y_pred=reg.predict(x_test)

In [ ]:
import math
from sklearn.metrics import r2_score
print("R2 score: ",r2_score(y_test,y_pred))